In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split


# importer
df =pd.read_csv("../data/cleaned_data.csv")
# Selecting parameters to train our price prediction on
df_selection = df[[ 
    "price",              
    "living_area",
    "building_type",          
    "terrace",             
    "garden",              
    "swimming_pool",       
    "energy_class"]]

#df_selection = df_selection.replace(["apartment", "house"], [0, 1])
#Switching price to log_price


#Dropping missing values in living_area and price
df_selection = df_selection.dropna(subset=["living_area", "price"])
df_selection['log_price'] = df_selection['price'].apply(np.log10)
df_selection = df_selection.drop(columns = ['price'])
#Fill missing values in swimming_pool with zeros.
df_selection["swimming_pool"] = df_selection["swimming_pool"].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(
    df_selection[["living_area",       
    "terrace",             
    "garden",              
    "swimming_pool",       
    "energy_class"]], 
    df_selection['log_price'], 
    test_size = 0.2)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)
df_preds = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': y_pred.squeeze()})
print(df_preds)


        Actual  Predicted
2699  5.525045   5.470227
4953  5.201397   5.503632
3819  5.352183   5.477737
6883  5.838849   5.605825
6985  6.254064   5.616586
...        ...        ...
1746  5.874482   5.536942
6697  5.378398   5.547771
470   5.190332   5.481641
7499  5.469822   5.506394
2957  5.475671   5.521337

[1835 rows x 2 columns]
